In [1]:
cd ../../../datasets/190829_Kanshan_zjfx

F:\Codes\datasets\190829_Kanshan_zjfx


In [2]:
ls

 驱动器 F 中的卷是 存储
 卷的序列号是 C14D-581B

 F:\Codes\datasets\190829_Kanshan_zjfx 的目录

2019/11/19  08:48    <DIR>          .
2019/11/19  08:48    <DIR>          ..
2019/09/26  19:08     5,805,965,112 answer_info_0926.txt
2019/11/08  14:33               188 assert.py
2019/09/26  18:10       333,836,065 invite_info_0926.txt
2019/09/28  12:14        37,860,903 invite_info_evaluate_1_0926.txt
2019/09/26  17:12       552,574,982 member_info_0926.txt
2019/11/08  21:28     1,144,346,559 question_info_0926.sql
2019/09/26  18:01     1,074,273,891 question_info_0926.txt
2019/10/18  16:35    <DIR>          sample
2019/08/05  17:19        15,951,125 single_word_vectors_64d.txt
2019/08/05  17:20        69,411,196 topic_vectors_64d.txt
2019/11/18  18:20     4,006,275,165 train.csv
2019/11/18  17:11     3,456,569,215 train.sql
2019/11/18  17:38       484,214,123 val.csv
2019/08/05  17:19     1,230,810,962 word_vectors_64d.txt
2019/11/06  09:59     9,418,974,739 zhihu2019_dataset.sql
              14 个文件 27,63

## 0.1 超参数

In [3]:
import numpy as np
import pandas as pd
# import pymysql
import torch
from tqdm import tqdm

BATCH_SIZE = 65536
DB_NAME = 'zhihu2019_dataset'
DO_DATASET_BALANCE = True
EPOCH = 3
LR = 0.07
TRAIN_ALL = True

PKL_DIR_READ = '../../jupyter/190919_DataMiningHW/project3/storage/train_param.pkl'
PKL_DIR_OUT = '../../jupyter/190919_DataMiningHW/project3/storage/train_param.pkl'
RESULT_DIR_OUT = '../../jupyter/190919_DataMiningHW/project3/storage/result.txt'
STORAGE_DIR = '../../jupyter/190919_DataMiningHW/project3/storage'

DEVICE = torch.device('cuda:3') if torch.cuda.is_available() else torch.device('cpu')
DEVICE

device(type='cpu')

## 1.1 加载数据集

取训练集。

In [4]:
# with DB() as db:
#     db.execute('select * from train')
#     train = pd.DataFrame(db.fetchall())

train = pd.read_csv('train.csv', header=0, sep=',')

取测试集。

In [5]:
# with DB() as db:
#     db.execute('select * from val')
#     test = pd.DataFrame(db.fetchall())

test = pd.read_csv('val.csv', header=0, sep=',')

合并二集。

In [6]:
col_list = ['q_id', 'u_id', 'do_answer', 'invite_hour', 'follow_topic_hit', 
             'interest_topic_hit', 'sex', 'access_freq', 'bin_feat_a', 'bin_feat_b', 
             'bin_feat_c', 'bin_feat_d', 'bin_feat_e', 'multi_feat_a', 'multi_feat_b',
             'multi_feat_c', 'multi_feat_d', 'multi_feat_e', 'salt', 'u_answer', 
             'u_good_answer', 'u_recommand_answer', 'u_image_answer', 'u_video_answer', 
             'u_word_avg', 'u_like_avg', 'u_unlike_avg', 'u_comment_avg', 
             'u_collect_avg', 'u_thanks_avg', 'u_report_avg','u_nohelp_avg', 
             'u_oppose_avg', 'q_answer', 'q_good_answer', 'q_recommand_answer',
             'q_image_answer', 'q_video_answer', 'q_word_avg', 'q_like', 'q_comment', 
             'q_collect', 'q_thanks']
# train = train[col_list]
# test = test[col_list]

In [7]:
data = pd.concat([train, test], axis=0)
data.columns

Index(['q_id', 'u_id', 'do_answer', 'invite_hour', 'follow_topic_hit',
       'interest_topic_hit', 'sex', 'access_freq', 'bin_feat_a', 'bin_feat_b',
       'bin_feat_c', 'bin_feat_d', 'bin_feat_e', 'multi_feat_a',
       'multi_feat_b', 'multi_feat_c', 'multi_feat_d', 'multi_feat_e', 'salt',
       'u_answer', 'u_good_answer', 'u_recommand_answer', 'u_image_answer',
       'u_video_answer', 'u_word_avg', 'u_like_avg', 'u_unlike_avg',
       'u_comment_avg', 'u_collect_avg', 'u_thanks_avg', 'u_report_avg',
       'u_nohelp_avg', 'u_oppose_avg', 'q_answer', 'q_good_answer',
       'q_recommand_answer', 'q_image_answer', 'q_video_answer', 'q_word_avg',
       'q_like', 'q_comment', 'q_collect', 'q_thanks'],
      dtype='object')

标准化数据

In [8]:
from sklearn.preprocessing import scale

for i in ['invite_hour', 'follow_topic_hit', 
             'interest_topic_hit', 'bin_feat_a', 'bin_feat_b', 
             'bin_feat_c', 'bin_feat_d', 'bin_feat_e', 'salt', 'u_answer', 
             'u_good_answer', 'u_recommand_answer', 'u_image_answer', 'u_video_answer', 
             'u_word_avg', 'u_like_avg', 'u_unlike_avg', 'u_comment_avg', 
             'u_collect_avg', 'u_thanks_avg', 'u_report_avg','u_nohelp_avg', 
             'u_oppose_avg', 'q_answer', 'q_good_answer', 'q_recommand_answer',
             'q_image_answer', 'q_video_answer', 'q_word_avg', 'q_like', 'q_comment', 
             'q_collect', 'q_thanks']:
    data[i] = scale(data[i].values)

## 1.1.5 清洗准备工作

计算数据相关性

In [36]:
for i in ['invite_hour', 'follow_topic_hit', 
             'interest_topic_hit', 'bin_feat_a', 'bin_feat_b', 
             'bin_feat_c', 'bin_feat_d', 'bin_feat_e', 'salt', 'u_answer', 
             'u_good_answer', 'u_recommand_answer', 'u_image_answer', 'u_video_answer', 
             'u_word_avg', 'u_like_avg', 'u_unlike_avg', 'u_comment_avg', 
             'u_collect_avg', 'u_thanks_avg', 'u_report_avg','u_nohelp_avg', 
             'u_oppose_avg', 'q_answer', 'q_good_answer', 'q_recommand_answer',
             'q_image_answer', 'q_video_answer', 'q_word_avg', 'q_like', 'q_comment', 
             'q_collect', 'q_thanks']:
    k1 = data['do_answer'].corr(data[i], method='pearson')
    k2 = data['do_answer'].corr(data[i], method='spearman')
    k3 = data['do_answer'].corr(data[i], method='kendall')
    print('%s\t\t%.4f\t%.4f\t%.4f' % (i, k1, k2, k3))

invite_hour		-0.0319	-0.0310	-0.0253
follow_topic_hit		0.0297	0.0265	0.0252
interest_topic_hit		0.0078	0.0080	0.0075
bin_feat_a		0.0147	0.0145	0.0141
bin_feat_b		-0.0117	-0.0123	-0.0120
bin_feat_c		-0.0082	-0.0085	-0.0082
bin_feat_d		0.0012	0.0010	0.0010
bin_feat_e		-0.0000	-0.0001	-0.0001
salt		0.0404	0.0526	0.0417
u_answer		0.1213	0.1572	0.1295
u_good_answer		0.0025	0.0039	0.0038
u_recommand_answer		0.0003	0.0002	0.0002
u_image_answer		-0.0121	0.0209	0.0190
u_video_answer		-0.0065	0.0101	0.0097
u_word_avg		-0.0024	0.0629	0.0509
u_like_avg		-0.0063	-0.0116	-0.0106
u_unlike_avg		-0.0075	-0.0163	-0.0156
u_comment_avg		-0.0083	0.0073	0.0067
u_collect_avg		-0.0028	-0.0118	-0.0112
u_thanks_avg		-0.0039	-0.0196	-0.0187
u_report_avg		-0.0005	-0.0004	-0.0004
u_nohelp_avg		-0.0045	-0.0089	-0.0086
u_oppose_avg		-0.0075	-0.0163	-0.0156
q_answer		0.0867	0.3735	0.3184
q_good_answer		0.0238	0.0249	0.0241
q_recommand_answer		0.0026	0.0029	0.0028
q_image_answer		0.0600	0.1606	0.1499
q_video_answer		0

## 1.2 处理数据集

清洗数据

In [9]:
stable_col_list = ['q_id', 'u_id', 'do_answer']
good_col_list = ['follow_topic_hit', 'interest_topic_hit', 'bin_feat_a',
                 'salt', 'u_answer', 'u_image_answer', 'u_video_answer', 
                 'u_word_avg', 'q_answer', 'q_good_answer',
                 'q_image_answer', 'q_video_answer', 'q_word_avg', 'q_like', 'q_comment', 
                 'q_collect', 'q_thanks']
good_bad_col_list = ['invite_hour', 'follow_topic_hit', 
             'interest_topic_hit', 'bin_feat_a', 'bin_feat_b', 
             'bin_feat_c', 'bin_feat_d', 'bin_feat_e', 'salt', 'u_answer', 
             'u_good_answer', 'u_recommand_answer', 'u_image_answer', 'u_video_answer', 
             'u_word_avg', 'u_like_avg', 'u_unlike_avg', 'u_comment_avg', 
             'u_collect_avg', 'u_thanks_avg', 'u_report_avg','u_nohelp_avg', 
             'u_oppose_avg', 'q_answer', 'q_good_answer', 'q_recommand_answer',
             'q_image_answer', 'q_video_answer', 'q_word_avg', 'q_like', 'q_comment', 
             'q_collect', 'q_thanks']
other_col_list = ['sex', 'access_freq', 'multi_feat_a', 'multi_feat_b', 
                  'multi_feat_c','multi_feat_d','multi_feat_e']
data = data[stable_col_list + good_bad_col_list + other_col_list]
data.columns

Index(['q_id', 'u_id', 'do_answer', 'invite_hour', 'follow_topic_hit',
       'interest_topic_hit', 'bin_feat_a', 'bin_feat_b', 'bin_feat_c',
       'bin_feat_d', 'bin_feat_e', 'salt', 'u_answer', 'u_good_answer',
       'u_recommand_answer', 'u_image_answer', 'u_video_answer', 'u_word_avg',
       'u_like_avg', 'u_unlike_avg', 'u_comment_avg', 'u_collect_avg',
       'u_thanks_avg', 'u_report_avg', 'u_nohelp_avg', 'u_oppose_avg',
       'q_answer', 'q_good_answer', 'q_recommand_answer', 'q_image_answer',
       'q_video_answer', 'q_word_avg', 'q_like', 'q_comment', 'q_collect',
       'q_thanks', 'sex', 'access_freq', 'multi_feat_a', 'multi_feat_b',
       'multi_feat_c', 'multi_feat_d', 'multi_feat_e'],
      dtype='object')

离散特征的`LabelEncoder`编码。

In [10]:
from sklearn.preprocessing import LabelEncoder

# 离散值编码
encoder = LabelEncoder()
for i in other_col_list:
    data[i] = encoder.fit_transform(data[i])
    
data.iloc[:10]

,q_id,u_id,do_answer,invite_hour,follow_topic_hit,interest_topic_hit,bin_feat_a,bin_feat_b,bin_feat_c,bin_feat_d,...,q_comment,q_collect,q_thanks,sex,access_freq,multi_feat_a,multi_feat_b,multi_feat_c,multi_feat_d,multi_feat_e
0,Q2166419046,M401693808,0,1.546644,1.313413,-0.274834,-1.404298,1.532982,-0.201699,-0.591582,...,-0.074245,-0.028820,-0.044241,2,4,1917,170,244,799,1
1,Q1550017551,M3392373099,0,-0.389192,-0.438017,-0.274834,0.712100,-0.652323,-0.201699,-0.591582,...,-0.070538,-0.028820,-0.044241,2,1,1092,115,249,657,0
2,Q604029601,M2317670257,0,0.314748,-0.438017,-0.274834,0.712100,-0.652323,-0.201699,-0.591582,...,-0.074245,-0.028820,-0.044241,2,4,1380,206,0,440,1
3,Q2350061229,M1618461867,0,-0.389192,-0.438017,2.528689,0.712100,-0.652323,-0.201699,1.690382,...,-0.063124,-0.028304,-0.041673,2,0,1092,115,244,799,1
4,Q2443223942,M3544409350,0,-1.621091,-0.438017,-0.274834,0.712100,-0.652323,-0.201699,-0.591582,...,-0.066831,-0.028820,-0.044241,2,1,506,204,175,705,1
5,Q640765464,M2818659842,0,0.490733,1.313413,-0.274834,0.712100,-0.652323,-0.201699,-0.591582,...,-0.074245,-0.028820,-0.044241,1,0,1380,206,0,440,1
6,Q795459266,M2818659842,0,1.194672,-0.438017,-0.274834,0.712100,-0.652323,-0.201699,-0.591582,...,-0.055711,-0.028820,-0.044241,1,0,1380,206,0,440,1
7,Q190554387,M1581217469,1,-0.917147,-0.438017,-0.274834,0.712100,-0.652323,-0.201699,-0.591582,...,-0.066831,-0.028820,-0.041673,2,1,1092,115,0,128,1
8,Q1958712851,M3021021791,0,1.018691,-0.438017,-0.274834,-1.404298,1.532982,-0.201699,-0.591582,...,-0.074245,-0.028820,-0.044241,2,4,1486,170,249,657,1
9,Q311993584,M1766315480,0,0.314748,-0.438017,-0.274834,0.712100,-0.652323,-0.201699,-0.591582,...,-0.048297,-0.028304,-0.036537,1,0,1092,115,370,1252,1


## 1.3 包装数据集

In [11]:
# import numpy as np

# if DO_DATASET_BALANCE:
#     temp = data[data['do_answer'] == 1]  # 用于均衡的数据
#     print('用于均衡的数据有 %d 条.' % len(temp))
#     train_x = data[data['do_answer']>-1].append([temp, temp, temp])
# else:
#     train_x = data[data['do_answer']>-1]
# train_x, train_y = train_x[col_list[3:]].values, train_x[['do_answer']].values[:, 0]

test_x = data[data['do_answer']==-1]
test_x, test_y = test_x[good_bad_col_list + other_col_list].values, test_x['do_answer'].values
# print('ok.')

In [12]:
# 正负样本df
positive_samples = data[data['do_answer'] == 1]
negative_samples = data[data['do_answer'] == 0]
print('pos_len:', len(positive_samples))
print('neg_len:', len(negative_samples))

pos_len: 1682914
neg_len: 7806219


In [13]:
from torch.utils.data import Dataset, DataLoader


class MyDataset(Dataset):
    """每次new都会随机均衡数据集。"""
    def __init__(self, mode='train'):
        self.mode = mode
        if mode == 'train':
            train_x = pd.concat([
                positive_samples,
                negative_samples.sample(n=len(positive_samples), axis=0)
            ], axis=0)
            train_x, train_y = train_x[good_col_list + other_col_list].values, train_x['do_answer'].values
            self.data = {
                'x': train_x,
                'y': train_y,
            }
        elif mode == 'test':
            self.data = {
                'x': test_x,
                'y': test_y,
            }
        else:
            raise
            
    def __getitem__(self, index):
        return self.data['x'][index], self.data['y'][index]
        
    def __len__(self):
        return len(self.data['x'])
    
    
print('ok.')

ok.


In [14]:
import math
from torch.utils.data import random_split

train_size, val_size, test_size = 0, 0, 0
train_len, val_len, test_len = 0, 0, 0
data_loader = {}

def flush_dataloader(log=True):
    global train_size, val_size, test_size, train_len, val_len, test_len, data_loader
    
    full_train_dataset = MyDataset(mode='train')
    test_dataset = MyDataset(mode='test')

    if TRAIN_ALL:
        data_loader = {
            'train': DataLoader(dataset=full_train_dataset, 
                                batch_size=BATCH_SIZE, 
                                num_workers=5,
                                shuffle=True),
            'test': DataLoader(dataset=test_dataset,
                               batch_size=BATCH_SIZE,
                               num_workers=5,
                               shuffle=False),
        }
        train_size = len(full_train_dataset)
        test_size = len(test_dataset)
    else:
        train_size = int(0.8 * len(full_train_dataset))
        val_size = len(full_train_dataset) - train_size
        train_dataset, val_dataset = random_split(full_train_dataset, [train_size, val_size])

        data_loader = {
            'train': DataLoader(dataset=train_dataset, 
                                batch_size=BATCH_SIZE, 
                               num_workers=5,
                                shuffle=True),
            'val': DataLoader(dataset=val_dataset, 
                              batch_size=BATCH_SIZE,
                               num_workers=5,
                              shuffle=False),
            'test': DataLoader(dataset=test_dataset,
                               batch_size=BATCH_SIZE,
                               num_workers=5,
                               shuffle=False),
        }
        train_size = len(train_dataset)
        val_size = len(val_dataset)
        test_size = len(test_dataset)

    if log:
        print('train_size:', train_size)
        print('val_size:', val_size)
        print('test_size:', test_size)

    # 计算一轮enumerate的长度
    train_len = math.ceil(train_size / BATCH_SIZE)
    val_len = math.ceil(val_size / BATCH_SIZE)
    test_len = math.ceil(test_size / BATCH_SIZE)

    if log:
        print('train_len:', train_len)
        print('val_len:', val_len)
        print('test_len:', test_len)

## 2.1 构建网络

In [47]:
import torch
import torch.nn.functional as F
from torch import nn


class MyNet_v2(nn.Module):
    def __init__(self, in_dim, out_dim=2, norm='dropout'):
        super(MyNet_v2, self).__init__()
        self.norm = norm

        in_dim = in_dim+1+1+4+2+2+3
        self.embed_sex = nn.Embedding(3, 2)
        self.embed_access_freq = nn.Embedding(5, 2)
        self.embed_multi_a = nn.Embedding(2561, 5)
        self.embed_multi_b = nn.Embedding(291, 3)
        self.embed_multi_c = nn.Embedding(428, 3)
        self.embed_multi_d = nn.Embedding(1556, 4)
        self.embed_multi_e = nn.Embedding(2, 1)
        self.fc1 = nn.Linear(in_dim, 512)
        self.fc2 = nn.Linear(512, 256)
        self.fc3 = nn.Linear(256, 128)
        self.fc4 = nn.Linear(128, out_dim)
        self.relu = nn.ReLU(inplace=True)
        if self.norm == 'batchnorm':
            self.bn0 = nn.BatchNorm1d(in_dim)
            self.bn1 = nn.BatchNorm1d(512)
            self.bn2 = nn.BatchNorm1d(256)
            self.bn3 = nn.BatchNorm1d(128)


    def forward(self, x):
        sex = self.embed_sex(x[:, -7].long())
        access_freq = self.embed_access_freq(x[:, -6].long())
        multi_a = self.embed_multi_a(x[:, -5].long())
        multi_b = self.embed_multi_b(x[:, -4].long())
        multi_c = self.embed_multi_c(x[:, -3].long())
        multi_d = self.embed_multi_d(x[:, -2].long())
        multi_e = self.embed_multi_e(x[:, -1].long())

        out = x[:, :-7]
        out = torch.cat(
            (out, sex, access_freq, multi_a, multi_b, multi_c, multi_d, multi_e), dim=1)
        
        if self.norm is None:
            out = self.fc1(out)
            out = self.fc2(self.relu(out))
            out = self.fc3(self.relu(out))
            out = self.fc4(self.relu(out))
        elif self.norm == 'dropout':
            out = self.fc1(out)
            out = F.dropout(out, p=0.5, training=self.training)
            out = self.fc2(self.relu(out))
            out = F.dropout(out, p=0.5, training=self.training)
            out = self.fc3(self.relu(out))
            out = F.dropout(out, p=0.5, training=self.training)
            out = self.fc4(self.relu(out))
        else:
            out = self.fc1(self.bn0(out))
            out = self.fc2(self.relu(self.bn1(out)))
            out = self.fc3(self.relu(self.bn2(out)))
            out = self.fc4(self.relu(self.bn3(out)))
        return out

In [48]:
# from my_model import MyNet_v2_1

net = MyNet_v2(len(good_col_list + other_col_list)).to(DEVICE)
net

MyNet_v2(
  (embed_sex): Embedding(3, 2)
  (embed_access_freq): Embedding(5, 2)
  (embed_multi_a): Embedding(2561, 5)
  (embed_multi_b): Embedding(291, 3)
  (embed_multi_c): Embedding(428, 3)
  (embed_multi_d): Embedding(1556, 4)
  (embed_multi_e): Embedding(2, 1)
  (fc1): Linear(in_features=37, out_features=512, bias=True)
  (fc2): Linear(in_features=512, out_features=256, bias=True)
  (fc3): Linear(in_features=256, out_features=128, bias=True)
  (fc4): Linear(in_features=128, out_features=2, bias=True)
  (relu): ReLU(inplace=True)
)

## 2.2 优化器、损失函数

In [57]:
EPOCH = 3
LR = 0.01

In [58]:
from torch import optim, nn

loss_function = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=LR, momentum=0.9)
# scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=2, gamma=0.75)

'ok'

'ok'

## 2.3 训练

In [59]:
from tqdm import tqdm

for epoch in range(EPOCH):
    count, running_correct, running_loss = 0, 0, 0
    flush_dataloader(log=False)
    
    net.train()
    with tqdm(total=train_len) as pbar:
        for step, (bx, by) in enumerate(data_loader['train']):
            # 训练
            bx = bx.float().to(DEVICE)
            by = by.long().to(DEVICE)

            prediction = net(bx)
            loss = loss_function(prediction, by)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            # 统计成效
            pre = torch.argmax(torch.softmax(prediction, dim=1), dim=1)
            my_loss = float(loss.data.cpu().numpy())
            my_correct = sum(pre.cpu().numpy() == by.cpu().numpy())
            my_acc = my_correct / BATCH_SIZE
            count += 1
            running_loss += my_loss
            running_correct += my_correct
            
            # 更新进度条
            pbar.update(1)
            pbar.set_description('Epoch: %d, Loss: %.4f, Acc: %.4f' % (
                                  epoch, 
                                  my_loss, 
                                  my_acc))
    
    # 输出一轮结果
    print('Epoch: %d, Loss: %.4f, Acc: %.4f, Lr: %.5f' % (
           epoch,
           running_loss / count,
           running_correct / train_size,
           optimizer.state_dict()['param_groups'][0]['lr']))
    
    # 保存参数
    torch.save(net.state_dict(), PKL_DIR_OUT)
    
    # lr scheduler
#     scheduler.step()

Epoch: 0, Loss: 0.6026, Acc: 0.2419: 100%|██████████| 52/52 [00:16<00:00,  3.24it/s]


Epoch: 0, Loss: 0.6021, Acc: 0.6743, Lr: 0.01000


Epoch: 1, Loss: 0.5999, Acc: 0.2412: 100%|██████████| 52/52 [00:15<00:00,  3.36it/s]


Epoch: 1, Loss: 0.6015, Acc: 0.6748, Lr: 0.01000


Epoch: 2, Loss: 0.6023, Acc: 0.2429: 100%|██████████| 52/52 [00:15<00:00,  3.28it/s]

Epoch: 2, Loss: 0.6012, Acc: 0.6749, Lr: 0.01000


## 2.4 存储网络

In [60]:
torch.save(net.state_dict(), PKL_DIR_OUT)

## 3.1 LightGBM分类

In [26]:
data[good_bad_col_list + other_col_list]

MemoryError: 

In [16]:
from lightgbm import LGBMClassifier

tmp = data[good_bad_col_list + other_col_list]
train_x = tmp[:train.shape[0]].values
train_y = data[:train.shape[0]]['do_answer'].values
test_x = tmp[train.shape[0]:].values
test_x.shape

MemoryError: 

In [14]:
model_lgb = LGBMClassifier(boosting_type='gbdt', num_leaves=64, learning_rate=0.01, n_estimators=2000,
                           max_bin=425, subsample_for_bin=50000, objective='binary', min_split_gain=0,
                           min_child_weight=5, min_child_samples=10, subsample=0.8, subsample_freq=1,
                           colsample_bytree=1, reg_alpha=3, reg_lambda=5, seed=1000, n_jobs=-1, silent=True)
model_lgb

LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=1,
               importance_type='split', learning_rate=0.01, max_bin=425,
               max_depth=-1, min_child_samples=10, min_child_weight=5,
               min_split_gain=0, n_estimators=2000, n_jobs=-1, num_leaves=64,
               objective='binary', random_state=None, reg_alpha=3, reg_lambda=5,
               seed=1000, silent=True, subsample=0.8, subsample_for_bin=50000,
               subsample_freq=1)

In [15]:
model_lgb.fit(train_x, train_y, 
                  eval_names=['train'],
                  eval_metric=['logloss','auc'],
                  eval_set=[(train_x, train_y)],
                  early_stopping_rounds=10)

[1]	train's auc: 0.798745	train's binary_logloss: 0.46561
Training until validation scores don't improve for 10 rounds
[2]	train's auc: 0.799619	train's binary_logloss: 0.463913
[3]	train's auc: 0.800084	train's binary_logloss: 0.462259
[4]	train's auc: 0.800877	train's binary_logloss: 0.460639
[5]	train's auc: 0.801082	train's binary_logloss: 0.459064
[6]	train's auc: 0.801163	train's binary_logloss: 0.457523
[7]	train's auc: 0.801276	train's binary_logloss: 0.45602
[8]	train's auc: 0.801307	train's binary_logloss: 0.454554
[9]	train's auc: 0.801458	train's binary_logloss: 0.45312
[10]	train's auc: 0.801632	train's binary_logloss: 0.451714
[11]	train's auc: 0.801739	train's binary_logloss: 0.450344
[12]	train's auc: 0.801896	train's binary_logloss: 0.449002
[13]	train's auc: 0.80217	train's binary_logloss: 0.447685
[14]	train's auc: 0.802392	train's binary_logloss: 0.446397
[15]	train's auc: 0.802493	train's binary_logloss: 0.445139
[16]	train's auc: 0.802579	train's binary_logloss: 0

[137]	train's auc: 0.814608	train's binary_logloss: 0.379427
[138]	train's auc: 0.814689	train's binary_logloss: 0.379211
[139]	train's auc: 0.814792	train's binary_logloss: 0.378992
[140]	train's auc: 0.814848	train's binary_logloss: 0.37879
[141]	train's auc: 0.814919	train's binary_logloss: 0.378581
[142]	train's auc: 0.815013	train's binary_logloss: 0.378369
[143]	train's auc: 0.815071	train's binary_logloss: 0.378175
[144]	train's auc: 0.815127	train's binary_logloss: 0.37798
[145]	train's auc: 0.815297	train's binary_logloss: 0.377752
[146]	train's auc: 0.815398	train's binary_logloss: 0.377545
[147]	train's auc: 0.815468	train's binary_logloss: 0.377356
[148]	train's auc: 0.815525	train's binary_logloss: 0.377172
[149]	train's auc: 0.815586	train's binary_logloss: 0.376985
[150]	train's auc: 0.815651	train's binary_logloss: 0.376799
[151]	train's auc: 0.815735	train's binary_logloss: 0.37661
[152]	train's auc: 0.815789	train's binary_logloss: 0.376434
[153]	train's auc: 0.815887

[272]	train's auc: 0.822873	train's binary_logloss: 0.363447
[273]	train's auc: 0.822884	train's binary_logloss: 0.363402
[274]	train's auc: 0.822948	train's binary_logloss: 0.363333
[275]	train's auc: 0.822989	train's binary_logloss: 0.363275
[276]	train's auc: 0.823027	train's binary_logloss: 0.363216
[277]	train's auc: 0.823065	train's binary_logloss: 0.363158
[278]	train's auc: 0.82308	train's binary_logloss: 0.363114
[279]	train's auc: 0.823096	train's binary_logloss: 0.36307
[280]	train's auc: 0.823136	train's binary_logloss: 0.363011
[281]	train's auc: 0.823209	train's binary_logloss: 0.362937
[282]	train's auc: 0.823246	train's binary_logloss: 0.362883
[283]	train's auc: 0.823304	train's binary_logloss: 0.362819
[284]	train's auc: 0.823319	train's binary_logloss: 0.362778
[285]	train's auc: 0.823337	train's binary_logloss: 0.362736
[286]	train's auc: 0.82338	train's binary_logloss: 0.362679
[287]	train's auc: 0.823402	train's binary_logloss: 0.362637
[288]	train's auc: 0.823432

[407]	train's auc: 0.825928	train's binary_logloss: 0.358941
[408]	train's auc: 0.825951	train's binary_logloss: 0.358915
[409]	train's auc: 0.825968	train's binary_logloss: 0.358897
[410]	train's auc: 0.825983	train's binary_logloss: 0.358879
[411]	train's auc: 0.826023	train's binary_logloss: 0.358844
[412]	train's auc: 0.826053	train's binary_logloss: 0.358815
[413]	train's auc: 0.826079	train's binary_logloss: 0.358788
[414]	train's auc: 0.826104	train's binary_logloss: 0.358761
[415]	train's auc: 0.826128	train's binary_logloss: 0.358735
[416]	train's auc: 0.826147	train's binary_logloss: 0.358713
[417]	train's auc: 0.826167	train's binary_logloss: 0.358693
[418]	train's auc: 0.826176	train's binary_logloss: 0.358678
[419]	train's auc: 0.826201	train's binary_logloss: 0.358653
[420]	train's auc: 0.826215	train's binary_logloss: 0.358637
[421]	train's auc: 0.826251	train's binary_logloss: 0.358605
[422]	train's auc: 0.826278	train's binary_logloss: 0.358579
[423]	train's auc: 0.826

[542]	train's auc: 0.828311	train's binary_logloss: 0.356583
[543]	train's auc: 0.828331	train's binary_logloss: 0.356567
[544]	train's auc: 0.828345	train's binary_logloss: 0.356552
[545]	train's auc: 0.828353	train's binary_logloss: 0.356544
[546]	train's auc: 0.828369	train's binary_logloss: 0.356528
[547]	train's auc: 0.828381	train's binary_logloss: 0.356516
[548]	train's auc: 0.828391	train's binary_logloss: 0.356505
[549]	train's auc: 0.828401	train's binary_logloss: 0.356494
[550]	train's auc: 0.828415	train's binary_logloss: 0.356481
[551]	train's auc: 0.828421	train's binary_logloss: 0.356473
[552]	train's auc: 0.828431	train's binary_logloss: 0.356462
[553]	train's auc: 0.828441	train's binary_logloss: 0.356453
[554]	train's auc: 0.828453	train's binary_logloss: 0.356442
[555]	train's auc: 0.828464	train's binary_logloss: 0.356431
[556]	train's auc: 0.828474	train's binary_logloss: 0.356421
[557]	train's auc: 0.828493	train's binary_logloss: 0.356404
[558]	train's auc: 0.828

...

[1480]	train's auc: 0.83389	train's binary_logloss: 0.351565
[1481]	train's auc: 0.833891	train's binary_logloss: 0.351564
[1482]	train's auc: 0.833903	train's binary_logloss: 0.351555
[1483]	train's auc: 0.833904	train's binary_logloss: 0.351553
[1484]	train's auc: 0.833911	train's binary_logloss: 0.351548
[1485]	train's auc: 0.833913	train's binary_logloss: 0.351546
[1486]	train's auc: 0.833915	train's binary_logloss: 0.351544
[1487]	train's auc: 0.833917	train's binary_logloss: 0.351543
[1488]	train's auc: 0.833919	train's binary_logloss: 0.351541
[1489]	train's auc: 0.833921	train's binary_logloss: 0.351539
[1490]	train's auc: 0.833922	train's binary_logloss: 0.351537
[1491]	train's auc: 0.833925	train's binary_logloss: 0.351535
[1492]	train's auc: 0.833927	train's binary_logloss: 0.351533
[1493]	train's auc: 0.83393	train's binary_logloss: 0.35153
[1494]	train's auc: 0.833932	train's binary_logloss: 0.351528
[1495]	train's auc: 0.833935	train's binary_logloss: 0.351526
[1496]	trai

[1613]	train's auc: 0.834222	train's binary_logloss: 0.351273
[1614]	train's auc: 0.834224	train's binary_logloss: 0.351272
[1615]	train's auc: 0.834226	train's binary_logloss: 0.351271
[1616]	train's auc: 0.834228	train's binary_logloss: 0.351269
[1617]	train's auc: 0.83423	train's binary_logloss: 0.351268
[1618]	train's auc: 0.834231	train's binary_logloss: 0.351266
[1619]	train's auc: 0.834234	train's binary_logloss: 0.351264
[1620]	train's auc: 0.834236	train's binary_logloss: 0.351262
[1621]	train's auc: 0.834239	train's binary_logloss: 0.35126
[1622]	train's auc: 0.83424	train's binary_logloss: 0.351258
[1623]	train's auc: 0.834242	train's binary_logloss: 0.351257
[1624]	train's auc: 0.834244	train's binary_logloss: 0.351255
[1625]	train's auc: 0.834246	train's binary_logloss: 0.351254
[1626]	train's auc: 0.834247	train's binary_logloss: 0.351253
[1627]	train's auc: 0.834262	train's binary_logloss: 0.351241
[1628]	train's auc: 0.834266	train's binary_logloss: 0.351238
[1629]	trai

[1746]	train's auc: 0.834541	train's binary_logloss: 0.350998
[1747]	train's auc: 0.834542	train's binary_logloss: 0.350997
[1748]	train's auc: 0.834546	train's binary_logloss: 0.350994
[1749]	train's auc: 0.834548	train's binary_logloss: 0.350992
[1750]	train's auc: 0.834549	train's binary_logloss: 0.350991
[1751]	train's auc: 0.83455	train's binary_logloss: 0.350989
[1752]	train's auc: 0.834553	train's binary_logloss: 0.350987
[1753]	train's auc: 0.834554	train's binary_logloss: 0.350986
[1754]	train's auc: 0.834556	train's binary_logloss: 0.350985
[1755]	train's auc: 0.834563	train's binary_logloss: 0.350979
[1756]	train's auc: 0.834565	train's binary_logloss: 0.350977
[1757]	train's auc: 0.834567	train's binary_logloss: 0.350975
[1758]	train's auc: 0.834569	train's binary_logloss: 0.350973
[1759]	train's auc: 0.834571	train's binary_logloss: 0.350971
[1760]	train's auc: 0.834573	train's binary_logloss: 0.35097
[1761]	train's auc: 0.834574	train's binary_logloss: 0.350969
[1762]	tra

[1879]	train's auc: 0.834878	train's binary_logloss: 0.350709
[1880]	train's auc: 0.834881	train's binary_logloss: 0.350706
[1881]	train's auc: 0.834886	train's binary_logloss: 0.350701
[1882]	train's auc: 0.834888	train's binary_logloss: 0.3507
[1883]	train's auc: 0.83489	train's binary_logloss: 0.350698
[1884]	train's auc: 0.834891	train's binary_logloss: 0.350697
[1885]	train's auc: 0.834893	train's binary_logloss: 0.350695
[1886]	train's auc: 0.834901	train's binary_logloss: 0.350688
[1887]	train's auc: 0.834903	train's binary_logloss: 0.350686
[1888]	train's auc: 0.834905	train's binary_logloss: 0.350684
[1889]	train's auc: 0.834906	train's binary_logloss: 0.350683
[1890]	train's auc: 0.834908	train's binary_logloss: 0.350681
[1891]	train's auc: 0.83491	train's binary_logloss: 0.35068
[1892]	train's auc: 0.834911	train's binary_logloss: 0.350678
[1893]	train's auc: 0.834912	train's binary_logloss: 0.350678
[1894]	train's auc: 0.834914	train's binary_logloss: 0.350676
[1895]	train'

LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=1,
               importance_type='split', learning_rate=0.01, max_bin=425,
               max_depth=-1, min_child_samples=10, min_child_weight=5,
               min_split_gain=0, n_estimators=2000, n_jobs=-1, num_leaves=64,
               objective='binary', random_state=None, reg_alpha=3, reg_lambda=5,
               seed=1000, silent=True, subsample=0.8, subsample_for_bin=50000,
               subsample_freq=1)

In [19]:
# 保存分类器
model_lgb.booster_.save_model('%s/lgbmodel_112010.txt' % STORAGE_DIR)  # 存入模型
# lgmodel = lgb.Booster(model_file='%s/lgbmodel_112010.txt' % STORAGE_DIR)  # 读出模型

In [20]:
y_pred = model_lgb.predict_proba(test_x)[:, 1]
y_pred

array([0.01563778, 0.00134601, 0.01341078, ..., 0.01722789, 0.02916311,
       0.00342512])

In [21]:
result = data[data['do_answer'] == -1]
result.loc[:, 'do_answer'] = y_pred
result.iloc[0]

/home/zengrui/miniconda3/lib/python3.7/site-packages/pandas/core/indexing.py:494: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


q_id                  Q1493039281
u_id                    M64135255
do_answer               0.0156378
follow_topic_hit        -0.438017
interest_topic_hit        1.37349
bin_feat_a                -1.4043
salt                    -0.490024
u_answer                -0.400849
u_image_answer            4.64295
u_video_answer          -0.103026
u_word_avg                3.14211
q_answer                -0.204937
q_good_answer          -0.0647244
q_image_answer          -0.153848
q_video_answer         -0.0639039
q_word_avg              -0.437234
q_like                 -0.0547829
q_comment              -0.0742448
q_collect              -0.0288203
q_thanks               -0.0442414
sex                             2
access_freq                     4
multi_feat_a                 1092
multi_feat_b                  115
multi_feat_c                  187
multi_feat_d                 1193
multi_feat_e                    1
Name: 0, dtype: object

In [22]:
invite_info_evaluate = pd.read_csv('invite_info_evaluate_1_0926.txt', 
                                   header=None, sep='\t')
invite_info_evaluate.columns = ['q_id1', 'u_id1', 'invite_time']

In [23]:
result_out = pd.concat([result[['q_id', 'u_id', 'do_answer']], invite_info_evaluate],
                       axis=1)
len(result_out)

1141683

In [24]:
result_out[['q_id', 'u_id', 'invite_time', 
        'do_answer']].to_csv(RESULT_DIR_OUT, index=False, header=False, sep='\t')  # 导出数据

'ok'

'ok'

## 4.1 导出 test 结果

In [61]:
result = data[data['do_answer'] == -1]
result.iloc[-1]

q_id                  Q2590286630
u_id                  M3168697168
do_answer                      -1
follow_topic_hit        -0.438017
interest_topic_hit      -0.274834
bin_feat_a                 0.7121
salt                      1.73948
u_answer                 -0.27225
u_image_answer            1.61585
u_video_answer          -0.103026
u_word_avg                1.47854
q_answer                -0.204937
q_good_answer          -0.0647244
q_image_answer          -0.153848
q_video_answer         -0.0639039
q_word_avg              -0.437234
q_like                 -0.0547829
q_comment              -0.0742448
q_collect              -0.0288203
q_thanks               -0.0442414
sex                             1
access_freq                     0
multi_feat_a                 1092
multi_feat_b                  115
multi_feat_c                    6
multi_feat_d                  980
multi_feat_e                    1
Name: 1141682, dtype: object

In [63]:
from tqdm import tqdm
# from my_model import MyNet_v2_2

net = MyNet_v2(len(good_col_list + other_col_list)).to(DEVICE)
if torch.cuda.is_available():
    net.load_state_dict(torch.load(PKL_DIR_READ))
else:
    net.load_state_dict(torch.load(PKL_DIR_READ, map_location='cpu'))
net.eval()

res_list = []
with tqdm(total=test_len) as pbar:
    for step, (bx, by) in enumerate(data_loader['test']):
        bx = bx.float().to(DEVICE)
        by = by.long().to(DEVICE)
        
        prediction = net(bx)
        
        # 计算结果
        pre = torch.argmax(torch.softmax(prediction, dim=1), dim=1)
        res_list.extend(pre.cpu().numpy())
        pbar.update(1)
        
result.loc[:, 'do_answer'] = res_list
result[['do_answer']] = result[['do_answer']].astype(int)  # 数据类型转换为int

'ok'

100%|██████████| 18/18 [00:03<00:00,  4.64it/s]


'ok'

In [64]:
invite_info_evaluate = pd.read_csv('invite_info_evaluate_1_0926.txt', 
                                   header=None, sep='\t')
invite_info_evaluate.columns = ['q_id1', 'u_id1', 'invite_time']

In [65]:
result_out = pd.concat([result[['q_id', 'u_id', 'do_answer']], invite_info_evaluate],
                       axis=1)
len(result_out)

1141683

In [66]:
result_out[['q_id', 'u_id', 'invite_time', 
        'do_answer']].to_csv(RESULT_DIR_OUT, index=False, header=False, sep='\t')  # 导出数据

'ok'

'ok'